In [ ]:
# https://gseapy.readthedocs.io/en/latest/gseapy_example.html

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import gseapy as gp

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
dbs = gp.get_library_name()


## Read in naming file

In [4]:
inputFile = 'data/omics_LLM_DF.tsv'

In [5]:
genesets_LLM = pd.read_csv(inputFile, sep = "\t") 

In [6]:
# genesets_LLM.GeneSetID.unique()

In [7]:
# Remove "HALLMARK_" prefix for MSigDB gene sets
genesets_LLM['GeneSetName'] = genesets_LLM['GeneSetName'].str.removeprefix("HALLMARK_").str.replace('_', ' ')


In [8]:
genesets_LLM.columns

Index(['Source', 'GeneSetID', 'GeneSetName', 'GeneList', 'n_Genes', 'LLM Name',
       'LLM Analysis', 'Score'],
      dtype='object')

In [9]:
columnsToKeep = ['Source', 'GeneSetID', 'GeneSetName', 'GeneList', 'n_Genes', 'LLM Name',
       'LLM Analysis', 'Score']
geneSeparator = " "

In [10]:
LLM_DF = genesets_LLM

In [11]:
nGeneSets = LLM_DF.shape[0]

In [12]:
nGeneSets

250

In [13]:
LLM_repeated_DF = LLM_DF[LLM_DF[['Source', 'GeneSetID', 'GeneSetName', 'GeneList']].duplicated() == True]


In [14]:
LLM_DF.shape

(250, 8)

## Go through each system and run Enrichr 

In [ ]:
# Each system will have multiple rows

In [15]:
additionalCols = ['Rank', 'Overlap', 'P-value', 'Adjusted P-value', 'Genes', 'GO term', 'GO ID' ]

In [16]:
expandedColumnsToKeep =columnsToKeep;
expandedColumnsToKeep.extend(additionalCols)

In [17]:
expanded_LLM_genes_DF = pd.DataFrame(columns=expandedColumnsToKeep)

In [18]:
genesCol = 'GeneList'

In [19]:
for geneSetInd in range(10): #range(nGeneSets):
    print(geneSetInd)
    geneSetRow = LLM_DF.iloc[geneSetInd]
    geneSetGenes = geneSetRow[genesCol].split(geneSeparator)

    # Perform enrichment analysis
    enr = gp.enrichr(gene_list=geneSetGenes, # or "./tests/data/gene_list.txt",
                 gene_sets=['GO_Biological_Process_2023'],
                 organism='human', # don't forget to set organism to the one you desired! e.g. Yeast
                 outdir=None, # don't write to disk
                )
    
    # threshold
    # sigRes_DF =  enr.results.loc[enr.results['Adjusted P-value'] <=0.1]
    sigRes_DF = enr.results # remove thresholding, because want a name for each one
    sigRes_DF = sigRes_DF.sort_values('Adjusted P-value')
    if sigRes_DF.shape[0] == 0:
        
        print([geneSetInd, ' has not enirchment'])
        #continue # skip to next gene set
        sigRes_DF.loc[0, ['GO term', 'GO ID', 'nothing']] = 'NaN'
        sigRes_DF.loc[0, ['Rank']]  = 1
        sigRes_DF.loc[0, ['Overlap']]  = 0
        sigRes_DF.loc[0, ['P-value']]  = 1
        sigRes_DF.loc[0, ['Adjusted P-value']]  = 1
        sigRes_DF.loc[0, ['Genes']]  = ''
        
    else:
        expandedCols = sigRes_DF["Term"].str.split('(', expand = True)
        
        if expandedCols.shape[1] == 3:
            sigRes_DF[['GO term', 'GO ID', 'nothing']] = expandedCols
        else: # ==2
            sigRes_DF[['GO term', 'GO ID']] = expandedCols

        sigRes_DF[['GO ID', 'nothing']] = sigRes_DF["GO ID"].str.split(')', expand = True)

        # Get ranking of enriched GO terms
        sigRes_DF['Rank'] = sigRes_DF.reset_index().index

    
    # Doing what should be a simple colbind 
    geneSetRow_DF = geneSetRow.to_frame().T
    geneSetRow_repeated_DF = geneSetRow_DF.loc[geneSetRow_DF.index.repeat(sigRes_DF.shape[0])]
    geneSetRow_repeated_DF = geneSetRow_repeated_DF.reset_index()
    
    # system_Row_sigRes = systemRow_repeated_DF.join(sigRes_DF)
    geneSetRowRow_sigRes = pd.concat([geneSetRow_repeated_DF, sigRes_DF],axis=1)
    
    # Doing what should be a simple  rowbind
    if expanded_LLM_genes_DF.shape[0] == 0:
        expanded_LLM_genes_DF = geneSetRowRow_sigRes[expandedColumnsToKeep]
    else:
        expanded_LLM_genes_DF =  pd.concat([expanded_LLM_genes_DF, geneSetRowRow_sigRes[expandedColumnsToKeep]],axis=0)
    

0
1
2
3
4
5
6
7
8
9


In [20]:
outputFile = 'data/omics_LLM_Enrichr_subset_DF.tsv' 

In [21]:
expanded_LLM_genes_DF.to_csv(outputFile, sep = "\t") # Too large 

In [22]:
expanded_LLM_genes_DF.shape

(12838, 15)

In [23]:
reduced_LLM_genes_DF = expanded_LLM_genes_DF.loc[expanded_LLM_genes_DF.reset_index().groupby(['Source', 'GeneSetID', 'GeneSetName', 'GeneList'])['Adjusted P-value'].idxmin()]

KeyError: '[3451, 4450, 5167, 6886, 7731, 9840, 10593, 11473] not in index'

In [ ]:
# Convert 'Adjusted P-value' to numeric and replace NaNs with 1
expanded_LLM_genes_DF['Adjusted P-value'] = pd.to_numeric(expanded_LLM_genes_DF['Adjusted P-value'], errors='coerce').fillna(1)

# Reset the index of the DataFrame and store it
reset_df = expanded_LLM_genes_DF.reset_index()

# Find the indices of the minimum 'Adjusted P-value' for each group in the reset DataFrame
min_indices = reset_df.groupby(['Source', 'GeneSetID', 'GeneSetName', 'GeneList'])['Adjusted P-value'].idxmin()

# Use these indices to select rows from the reset DataFrame
selected_rows = reset_df.loc[min_indices]

# Use the original index from the selected rows to get rows from the original DataFrame
reduced_LLM_genes_DF = expanded_LLM_genes_DF.loc[selected_rows['index']]

# Now the reduced_LLM_genes_DF should have the rows based on minimum 'Adjusted P-value'


In [ ]:
reduced_LLM_genes_DF = expanded_LLM_genes_DF.loc[selected_rows['index']]


In [ ]:
reduced_LLM_genes_DF.shape

In [ ]:
reduced_LLM_genes_DF.to_csv('data/omics_LLM_Enrichr_reduced_DF.tsv' , sep = "\t")

In [ ]:
set(LLM_DF.GeneSetID.to_list()).difference(set(reduced_LLM_genes_DF.GeneSetID.to_list()))

In [ ]:
reduced_LLM_genes_DF = expanded_LLM_genes_DF.loc[expanded_LLM_genes_DF.reset_index().groupby(['Source', 'GeneSetID', 'GeneSetName', 'GeneList'])['Adjusted P-value'].idxmin()]